In [1]:
#Limpiza nueva

In [2]:
!pip install --upgrade spacy
!python -m spacy download es_core_news_sm
import spacy

# Cargar el modelo de idioma español de SpaCy
nlp = spacy.load('es_core_news_sm')

import spacy
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize.toktok import ToktokTokenizer
import re
import unicodedata

# Inicializar el tokenizador
tokenizer = ToktokTokenizer()

# Descargar la lista de stopwords en español de NLTK
nltk.download('stopwords')
stopword_list = nltk.corpus.stopwords.words('spanish')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 2.9 MB/s eta 0:00:0000:0100:01


✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


[nltk_data] Downloading package stopwords to /Users/sil/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
df = pd.read_csv("../data/capi_chat.csv")
df

,Fecha,Hora,Persona,Mensaje
0,"30/11/23,",19:12:50,Capibaras Ironhack,‎Los mensajes y las llamadas están cifrados de...
1,"30/11/23,",19:12:50,Alana Ironhack,‎Alana Ironhack creó el grupo “Capibaras Ironh...
2,"30/11/23,",19:12:50,Capibaras Ironhack,‎Alana Ironhack te añadió.
3,"30/11/23,",19:12:56,Alana Ironhack,Hola
4,"30/11/23,",19:13:12,Alana Ironhack,Les paso las fotos de ayer prepárense xq es un...
...,...,...,...,...
1769,"17/2/24,",21:09:53,Sil🐨,No y que también pensaba que esos días que íba...
1770,"17/2/24,",21:11:04,Alana Ironhack,Jajaj
1771,"17/2/24,",21:11:18,Alana Ironhack,Disfrutar de tu libertad
1772,"17/2/24,",21:18:17,Sil🐨,JAJAJAJAJA


In [4]:
# diccionario con el valor y nombre de los dias
week = {
    6:'Domingo',
    0:'Lunes',
    1:'Martes',
    2:'Miercoles',
    3:'Jueves',
    4:'Viernes',
    5:'Sabado'
}

# Diccionario con el valor y nombre de los meses
month = {
    1:'Ene',
    2:'Feb',
    3:'Mar',
    4:'Abr',
    5:'May',
    6:'Jun',
    7:'Jul',
    8:'Ago',
    9:'Sept',
    10:'Obt',
    11:'Nov',
    12:'Dic'
}


# dar formato de fecha a los dias 
df['Fecha'] = pd.to_datetime(df['Fecha'], dayfirst=True)

# nueva columna con los nombres de los dias
#df['Dia'] = df['Fecha'].dt.strftime('%A') #(esto tambien lo cambia)
df['Dia'] = df['Fecha'].dt.weekday.map(week)

# nueva columna con el numero del dia
df['Num_Day'] = df['Fecha'].dt.day

# nueva columna con el año 
df['Year'] = df['Fecha'].dt.year

# nueva columna con el numero de mes
df['Num_Month'] = df['Fecha'].dt.month

# nueva columna con el nombre de los meses
df['Month'] = df['Fecha'].dt.month.map(month)

# organizar las columnas
df = df[['Fecha', 'Dia', 'Num_Day','Num_Month', 'Month', 'Year', 'Hora', 'Persona', 'Mensaje']]

# cambiar el tipo de dato de la columna Day
df['Dia'] = df['Dia'].astype('category')
#cambiar formato fecha 
df['Fecha'] = df['Fecha'].dt.strftime('%d/%m/%Y')
df

/var/folders/1m/9tdt_k994v175ff72qvvlr7r0000gn/T/ipykernel_97235/2407361247.py:30: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Fecha'] = pd.to_datetime(df['Fecha'], dayfirst=True)


,Fecha,Dia,Num_Day,Num_Month,Month,Year,Hora,Persona,Mensaje
0,30/11/2023,Jueves,30,11,Nov,2023,19:12:50,Capibaras Ironhack,‎Los mensajes y las llamadas están cifrados de...
1,30/11/2023,Jueves,30,11,Nov,2023,19:12:50,Alana Ironhack,‎Alana Ironhack creó el grupo “Capibaras Ironh...
2,30/11/2023,Jueves,30,11,Nov,2023,19:12:50,Capibaras Ironhack,‎Alana Ironhack te añadió.
3,30/11/2023,Jueves,30,11,Nov,2023,19:12:56,Alana Ironhack,Hola
4,30/11/2023,Jueves,30,11,Nov,2023,19:13:12,Alana Ironhack,Les paso las fotos de ayer prepárense xq es un...
...,...,...,...,...,...,...,...,...,...
1769,17/02/2024,Sabado,17,2,Feb,2024,21:09:53,Sil🐨,No y que también pensaba que esos días que íba...
1770,17/02/2024,Sabado,17,2,Feb,2024,21:11:04,Alana Ironhack,Jajaj
1771,17/02/2024,Sabado,17,2,Feb,2024,21:11:18,Alana Ironhack,Disfrutar de tu libertad
1772,17/02/2024,Sabado,17,2,Feb,2024,21:18:17,Sil🐨,JAJAJAJAJA


# Eliminamos acentos

In [5]:
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

remove_accented_chars('Sómě Áccěntěd těxt')

'Some Accented text'

# cambiamos abreviaturas

In [6]:
# abreviaturas

CONTRACCIONES_ESP = {
"xq": "porque",
"Holiiii": "Hola",
"sticker omitido": "imagen",
"imagen omitida": "imagen",
"q": "que",
"X": "por",
"x": "por",
"esq": "es que",
"GIF omitido": "GIF",
"Valee": "vale",
"perfe": "perfecto",
"Siiii": "Si",
"siii": "Si",
"Ayyy":"Ay",
"proque": "porque",
"nose": "no se",
"tmbn": "tambien",
"lab": "laboratorio",
"labs": "labboratotios",
"lst": "lista",
"reu": "reunion",
"michi": "gato",
"jaajaaj": "jajajajaja",
"xd": "risa",
"capiiis": "capi",
"smpr": "siempre",
"Noup": "No",
"okii": "vale",
"year": "año",
"string": "letras",
"colorinchis": "colores",
"pull request": "subir",
"LOL": "risa a carcajadas",
"Wsp": "WhatsApp",
"Wasap": "WhatsApp",
"wa": "WhatsApp",
"Guasap": "WhatsApp",
"Pq": "porque",
"Xq": "porque",
"q": "que",
"Dnd": "donde",
"K": "que",
"T": "te",
"+/-": "mas o menos",
"Nd": "nada",
"Vdd": "verdad",
"Ntp": "no te preocupes",
"Tqm": "Te quiero mucho",
"Tq": "Te quiero",
"Tmb": "tambien",
"Dps": "despues",
"Xfa": "por favor",
"Dd": "donde",
"Oki": "vale",
"Ok": "vale",
"Tkt": "tranquilo",
"Tqg": "te quiero un monton",
"Bss": "besos",
"Tmbn": "tambien",
"Fvr": "favor",
"Msj": "mensaje",
"LMAO": "risa",
"BRB": "Vuelvo enseguida",
"IDK": "no se",
"idk": "no se",
"ASAP": "tan pronto como sea posible",
"OMG": "Oh Dios mio",
"BFF": "Mejores amigos para siempre",
"Okey": "vale"
    }

In [7]:
def corregir_abreviaturas(texto, mapeo_abreviaturas=CONTRACCIONES_ESP):
    texto_corregido = []
    for palabra in texto.replace(".","").split():
        palabra_corregida = mapeo_abreviaturas.get(palabra.lower(), palabra)
        if palabra_corregida is not None:
            texto_corregido.append(palabra_corregida)
        else:
            texto_corregido.append(palabra)
    return ' '.join(texto_corregido)
# Llamar a la función corregir_abreviaturas con el diccionario de abreviaturas
a = corregir_abreviaturas( 'Q tal. okey xq.')
print(a)

que tal okey porque


# caracteres especiales

In [8]:
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

remove_special_characters("¡Bueno, esto fue divertido! ¿Qué opinas? 123#@!", 
                          remove_digits=True)

'Bueno esto fue divertido Qu opinas '

# Text lemmatization

In [9]:
# Importar NLTK
import nltk
from nltk.stem import SnowballStemmer

# Crear una instancia de SnowballStemmer para español
stemmer_es = SnowballStemmer('spanish')

def lemmatize_text(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

lemmatize_text('Mi sistema sigue fallando. El suyo falló ayer, el nuestro falla diariamente.')

'mi sistema seguir fallar . el suyo fallar ayer , el nuestro falla diariamente .'

# Text stemming

In [10]:
def simple_stemmer(text):
    ps = nltk.porter.PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

simple_stemmer('Mi sistema sigue fallando. El suyo falló ayer, el nuestro falla diariamente.')

'mi sistema sigu fallando. el suyo falló ayer, el nuestro falla diariamente.'

# Remove stopwords

In [11]:
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

remove_stopwords("De, y, si son palabras vacías, la computadora no lo es")

', , si palabras vacías , computadora'

# Building a text normalizer

In [12]:
def normalize_corpus(corpus, contraction_expansion=True,
                     accented_char_removal=True, text_lower_case=True, 
                     text_lemmatization=True, special_char_removal=True, 
                     stopword_removal=True, remove_digits=True):
    
    normalized_corpus = []
    # normalize each document in the corpus
    for doc in corpus:
        # remove accented characters
        if accented_char_removal:
            doc = remove_accented_chars(doc)
        # expand contractions    
        if contraction_expansion:
            doc = corregir_abreviaturas(doc)
        # lowercase the text    
        if text_lower_case:
            doc = doc.lower()
        # remove extra newlines
        doc = re.sub(r'[\r|\n|\r\n]+', ' ',doc)
        # lemmatize text
        if text_lemmatization:
            doc = lemmatize_text(doc)
        # remove special characters and\or digits    
        if special_char_removal:
            # insert spaces between special characters to isolate them    
            special_char_pattern = re.compile(r'([{.(-)!}])')
            doc = special_char_pattern.sub(" \\1 ", doc)
            doc = remove_special_characters(doc, remove_digits=remove_digits)  
        # remove extra whitespace
        doc = re.sub(' +', ' ', doc)
        # remove stopwords
        if stopword_removal:
            doc = remove_stopwords(doc, is_lower_case=text_lower_case)
            
        normalized_corpus.append(doc)
        
    return normalized_corpus


# Pre-process and normalize news articles

In [13]:
df["Mensaje"]

0       ‎Los mensajes y las llamadas están cifrados de...
1       ‎Alana Ironhack creó el grupo “Capibaras Ironh...
2                              ‎Alana Ironhack te añadió.
3                                                    Hola
4       Les paso las fotos de ayer prepárense xq es un...
                              ...                        
1769    No y que también pensaba que esos días que íba...
1770                                                Jajaj
1771                             Disfrutar de tu libertad
1772                                           JAJAJAJAJA
1773                                                  Así
Name: Mensaje, Length: 1774, dtype: object

In [14]:
df['clean_text'] = normalize_corpus(df['Mensaje'])
#norm_corpus = list(df['Mensaje'])
df.iloc[1][['Mensaje', 'clean_text']].to_dict()
#news_df.iloc[1]['clean_text'].to_dict()
df

,Fecha,Dia,Num_Day,Num_Month,Month,Year,Hora,Persona,Mensaje,clean_text
0,30/11/2023,Jueves,30,11,Nov,2023,19:12:50,Capibaras Ironhack,‎Los mensajes y las llamadas están cifrados de...,mensaje llamada cifrado extremo extremo nadie ...
1,30/11/2023,Jueves,30,11,Nov,2023,19:12:50,Alana Ironhack,‎Alana Ironhack creó el grupo “Capibaras Ironh...,aln ironhack creer grupo capibara ironhack
2,30/11/2023,Jueves,30,11,Nov,2023,19:12:50,Capibaras Ironhack,‎Alana Ironhack te añadió.,aln ironhack t anadio
3,30/11/2023,Jueves,30,11,Nov,2023,19:12:56,Alana Ironhack,Hola,holar
4,30/11/2023,Jueves,30,11,Nov,2023,19:13:12,Alana Ironhack,Les paso las fotos de ayer prepárense xq es un...,l pasar foto ayer preparense ser bombardeo
...,...,...,...,...,...,...,...,...,...,...
1769,17/02/2024,Sabado,17,2,Feb,2024,21:09:53,Sil🐨,No y que también pensaba que esos días que íba...,tambien pensar dia ibir hacer
1770,17/02/2024,Sabado,17,2,Feb,2024,21:11:04,Alana Ironhack,Jajaj,jajaj
1771,17/02/2024,Sabado,17,2,Feb,2024,21:11:18,Alana Ironhack,Disfrutar de tu libertad,disfrutar libertad
1772,17/02/2024,Sabado,17,2,Feb,2024,21:18:17,Sil🐨,JAJAJAJAJA,jajajajajar


# exportar datos limpios

In [15]:
# Exportamos los datos limpios en un archivo csv
df.to_csv('../data/data_capi_chat_clear.csv', header=True, index=False)

# Dividir por conversaciones

In [16]:
# Convertir la columna 'Fecha' y 'Hora' a un objeto de fecha y hora combinado
df['timestamp'] = pd.to_datetime(df['Fecha'] + ' ' + df['Hora'])

# Definir el intervalo de tiempo para agrupar (por ejemplo, 10 minutos)
intervalo_tiempo = pd.Timedelta(minutes=10)

# Agrupar los mensajes en conversaciones basadas en el intervalo de tiempo
conversaciones_10 = df.groupby(pd.Grouper(key='timestamp', freq=intervalo_tiempo)).agg({'Persona': 'unique', 'Mensaje': ' '.join, 'clean_text': ' '.join}).reset_index()
conversaciones_10.head(10)

/var/folders/1m/9tdt_k994v175ff72qvvlr7r0000gn/T/ipykernel_97235/3567138705.py:2: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['timestamp'] = pd.to_datetime(df['Fecha'] + ' ' + df['Hora'])


,timestamp,Persona,Mensaje,clean_text
0,2023-11-30 19:10:00,"[Capibaras Ironhack, Alana Ironhack]",‎Los mensajes y las llamadas están cifrados de...,mensaje llamada cifrado extremo extremo nadie ...
1,2023-11-30 19:20:00,[],,
2,2023-11-30 19:30:00,[],,
3,2023-11-30 19:40:00,[],,
4,2023-11-30 19:50:00,[],,
5,2023-11-30 20:00:00,[],,
6,2023-11-30 20:10:00,[],,
7,2023-11-30 20:20:00,"[~ Teresa, Alana Ironhack]",Es un bombardeo que nos ahorra prácticamente u...,ser bombardeo ahorrar practicamente laboratori...
8,2023-11-30 20:30:00,[Sil🐨],Muchas gracias!,gracia
9,2023-11-30 20:40:00,[],,


In [17]:
# Exportamos los datos conversacion por 10 min
df.to_csv('../data/capi_chat_clear10min.csv', header=True, index=False)

In [18]:
# Convertir la columna 'Fecha' y 'Hora' a un objeto de fecha y hora combinado
df['timestamp'] = pd.to_datetime(df['Fecha'] + ' ' + df['Hora'])

# Definir el intervalo de tiempo para agrupar (por ejemplo, 10 minutos)
intervalo_tiempo = pd.Timedelta(minutes=20)

# Agrupar los mensajes en conversaciones basadas en el intervalo de tiempo
conversaciones_20 = df.groupby(pd.Grouper(key='timestamp', freq=intervalo_tiempo)).agg({'Persona': 'unique', 'Mensaje': ' '.join, 'clean_text': ' '.join}).reset_index()
conversaciones_20.head(10)

/var/folders/1m/9tdt_k994v175ff72qvvlr7r0000gn/T/ipykernel_97235/2730109981.py:2: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['timestamp'] = pd.to_datetime(df['Fecha'] + ' ' + df['Hora'])


,timestamp,Persona,Mensaje,clean_text
0,2023-11-30 19:00:00,"[Capibaras Ironhack, Alana Ironhack]",‎Los mensajes y las llamadas están cifrados de...,mensaje llamada cifrado extremo extremo nadie ...
1,2023-11-30 19:20:00,[],,
2,2023-11-30 19:40:00,[],,
3,2023-11-30 20:00:00,[],,
4,2023-11-30 20:20:00,"[~ Teresa, Alana Ironhack, Sil🐨]",Es un bombardeo que nos ahorra prácticamente u...,ser bombardeo ahorrar practicamente laboratori...
5,2023-11-30 20:40:00,[],,
6,2023-11-30 21:00:00,[],,
7,2023-11-30 21:20:00,[],,
8,2023-11-30 21:40:00,[],,
9,2023-11-30 22:00:00,[],,


In [19]:
# Exportamos los datos conversacion por 20 min
df.to_csv('../data/capi_chat_clear20min.csv', header=True, index=False)

In [20]:
# Convertir la columna 'Fecha' y 'Hora' a un objeto de fecha y hora combinado
df['timestamp'] = pd.to_datetime(df['Fecha'] + ' ' + df['Hora'])

# Definir el intervalo de tiempo para agrupar (por ejemplo, 10 minutos)
intervalo_tiempo = pd.Timedelta(minutes=30)

# Agrupar los mensajes en conversaciones basadas en el intervalo de tiempo
conversaciones_30 = df.groupby(pd.Grouper(key='timestamp', freq=intervalo_tiempo)).agg({'Persona': 'unique', 'Mensaje': ' '.join, 'clean_text': ' '.join}).reset_index()
conversaciones_30.head(10)

/var/folders/1m/9tdt_k994v175ff72qvvlr7r0000gn/T/ipykernel_97235/2964334069.py:2: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['timestamp'] = pd.to_datetime(df['Fecha'] + ' ' + df['Hora'])


,timestamp,Persona,Mensaje,clean_text
0,2023-11-30 19:00:00,"[Capibaras Ironhack, Alana Ironhack]",‎Los mensajes y las llamadas están cifrados de...,mensaje llamada cifrado extremo extremo nadie ...
1,2023-11-30 19:30:00,[],,
2,2023-11-30 20:00:00,"[~ Teresa, Alana Ironhack]",Es un bombardeo que nos ahorra prácticamente u...,ser bombardeo ahorrar practicamente laboratori...
3,2023-11-30 20:30:00,[Sil🐨],Muchas gracias!,gracia
4,2023-11-30 21:00:00,[],,
5,2023-11-30 21:30:00,[],,
6,2023-11-30 22:00:00,[],,
7,2023-11-30 22:30:00,[],,
8,2023-11-30 23:00:00,[],,
9,2023-11-30 23:30:00,[],,


In [21]:
# Exportamos los datos conversacion por 30 min
df.to_csv('../data/capi_chat_clear30min.csv', header=True, index=False)

# analisis con TextBlob

In [22]:
from textblob import TextBlob

In [38]:
mensajes = df['clean_text'].tolist()

# Lista para almacenar los sentimientos de cada mensaje
sentimientos = []

# Iterar sobre cada mensaje y calcular el sentimiento
for texto in mensajes:
    blob = TextBlob(texto)
    sentimiento = blob.sentiment.polarity
    if sentimiento > 0:
        sentimientos.append("positivo")
    elif sentimiento < 0:
        sentimientos.append("negativo")
    else:
        sentimientos.append("neutro")

# Imprimir los mensajes y sus sentimientos correspondientes
for i, mensaje in enumerate(mensajes):
    #print("Mensaje:", mensaje)
    #print("Sentimiento:", sentimientos[i])
    #print()
    # Agregar los sentimientos al DataFrame original
    df['texBlod'] = sentimientos


df

,Fecha,Dia,Num_Day,Num_Month,Month,Year,Hora,Persona,Mensaje,clean_text,timestamp,texBlod,sentimiento_vader,n-grams
0,30/11/2023,Jueves,30,11,Nov,2023,19:12:50,Capibaras Ironhack,‎Los mensajes y las llamadas están cifrados de...,mensaje llamada cifrado extremo extremo nadie ...,2023-11-30 19:12:50,neutro,neutro,neutro
1,30/11/2023,Jueves,30,11,Nov,2023,19:12:50,Alana Ironhack,‎Alana Ironhack creó el grupo “Capibaras Ironh...,aln ironhack creer grupo capibara ironhack,2023-11-30 19:12:50,neutro,neutro,neutro
2,30/11/2023,Jueves,30,11,Nov,2023,19:12:50,Capibaras Ironhack,‎Alana Ironhack te añadió.,aln ironhack t anadio,2023-11-30 19:12:50,neutro,neutro,neutro
3,30/11/2023,Jueves,30,11,Nov,2023,19:12:56,Alana Ironhack,Hola,holar,2023-11-30 19:12:56,neutro,neutro,neutro
4,30/11/2023,Jueves,30,11,Nov,2023,19:13:12,Alana Ironhack,Les paso las fotos de ayer prepárense xq es un...,l pasar foto ayer preparense ser bombardeo,2023-11-30 19:13:12,neutro,neutro,neutro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1769,17/02/2024,Sabado,17,2,Feb,2024,21:09:53,Sil🐨,No y que también pensaba que esos días que íba...,tambien pensar dia ibir hacer,2024-02-17 21:09:53,neutro,neutro,neutro
1770,17/02/2024,Sabado,17,2,Feb,2024,21:11:04,Alana Ironhack,Jajaj,jajaj,2024-02-17 21:11:04,neutro,neutro,neutro
1771,17/02/2024,Sabado,17,2,Feb,2024,21:11:18,Alana Ironhack,Disfrutar de tu libertad,disfrutar libertad,2024-02-17 21:11:18,neutro,neutro,neutro
1772,17/02/2024,Sabado,17,2,Feb,2024,21:18:17,Sil🐨,JAJAJAJAJA,jajajajajar,2024-02-17 21:18:17,neutro,neutro,neutro


In [51]:
# Lista para almacenar los sentimientos de cada mensaje
sentimientos = {"positivo": 0, "negativo": 0, "neutro": 0}

# Iterar sobre cada mensaje y calcular el sentimiento
for texto in mensajes:
    blob = TextBlob(texto)
    sentimiento = blob.sentiment.polarity
    if sentimiento > 0:
        sentimientos["positivo"] += 1
        #print("---->", texto)
    elif sentimiento < 0:
        sentimientos["negativo"] += 1
        #print("---->", texto)
    else:
        sentimientos["neutro"] += 1
        #print("---->", texto)

# Imprimir el recuento de sentimientos
for sentimiento, count in sentimientos.items():
    print(f"Sentimiento {sentimiento}: {count}")

Sentimiento positivo: 25
Sentimiento negativo: 14
Sentimiento neutro: 1735


In [45]:
sentimientos

{'positivo': 25, 'negativo': 14, 'neutro': 1735}

# vamos a probar con VADER

In [25]:
valores_de_sentimiento = {
    "feliz": 0.7,
    "bien": 0.5,
    "genial": 0.8,
    "triste": -0.7,
    "cabreado": -0.8,
    "mal": -0.5,
    "espero": 0,
    "caer": -0.1,
    "gripe": -0.1,
    "malo": -0.2,
    "vale perfecto": 0.8,
    "jajajajaja": 0.1,
}

In [27]:
# Define una función para calcular un resumen del análisis de sentimientos
def calcular_sentimiento_vader(texto):
    # Asegurarse de que el texto sea una cadena de texto
    if isinstance(texto, str):
        ### Analizando el sentimiento con VADER
        sia = SentimentIntensityAnalyzer()
        # Calcular los sentimientos utilizando VADER
        sentimientos = sia.polarity_scores(texto)
        # Devolver el sentimiento dominante
        if sentimientos['compound'] >= 0.05:
            return 'positivo'
        elif sentimientos['compound'] <= -0.05:
            return 'negativo'
        else:
            return 'neutro'
    else:
        return 'no aplicable'  # O cualquier otro valor que desees para casos donde el texto no sea una cadena de texto

# Aplicar la función a cada mensaje en el DataFrame
df['sentimiento_vader'] = df['clean_text'].apply(calcular_sentimiento_vader)
df

,Fecha,Dia,Num_Day,Num_Month,Month,Year,Hora,Persona,Mensaje,clean_text,timestamp,texBlod,sentimiento_vader
0,30/11/2023,Jueves,30,11,Nov,2023,19:12:50,Capibaras Ironhack,‎Los mensajes y las llamadas están cifrados de...,mensaje llamada cifrado extremo extremo nadie ...,2023-11-30 19:12:50,neutro,neutro
1,30/11/2023,Jueves,30,11,Nov,2023,19:12:50,Alana Ironhack,‎Alana Ironhack creó el grupo “Capibaras Ironh...,aln ironhack creer grupo capibara ironhack,2023-11-30 19:12:50,neutro,neutro
2,30/11/2023,Jueves,30,11,Nov,2023,19:12:50,Capibaras Ironhack,‎Alana Ironhack te añadió.,aln ironhack t anadio,2023-11-30 19:12:50,neutro,neutro
3,30/11/2023,Jueves,30,11,Nov,2023,19:12:56,Alana Ironhack,Hola,holar,2023-11-30 19:12:56,neutro,neutro
4,30/11/2023,Jueves,30,11,Nov,2023,19:13:12,Alana Ironhack,Les paso las fotos de ayer prepárense xq es un...,l pasar foto ayer preparense ser bombardeo,2023-11-30 19:13:12,neutro,neutro
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1769,17/02/2024,Sabado,17,2,Feb,2024,21:09:53,Sil🐨,No y que también pensaba que esos días que íba...,tambien pensar dia ibir hacer,2024-02-17 21:09:53,neutro,neutro
1770,17/02/2024,Sabado,17,2,Feb,2024,21:11:04,Alana Ironhack,Jajaj,jajaj,2024-02-17 21:11:04,neutro,neutro
1771,17/02/2024,Sabado,17,2,Feb,2024,21:11:18,Alana Ironhack,Disfrutar de tu libertad,disfrutar libertad,2024-02-17 21:11:18,neutro,neutro
1772,17/02/2024,Sabado,17,2,Feb,2024,21:18:17,Sil🐨,JAJAJAJAJA,jajajajajar,2024-02-17 21:18:17,neutro,neutro


# N-grams

In [28]:
import nltk, re, string, collections
from nltk.util import ngrams # function for making ngrams

import nltk
from nltk import word_tokenize
nltk.download('punkt')


[nltk_data] Downloading package punkt to /Users/sil/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [29]:
df['clean_text']

0       mensaje llamada cifrado extremo extremo nadie ...
1              aln ironhack creer grupo capibara ironhack
2                                   aln ironhack t anadio
3                                                   holar
4              l pasar foto ayer preparense ser bombardeo
                              ...                        
1769                        tambien pensar dia ibir hacer
1770                                                jajaj
1771                                   disfrutar libertad
1772                                          jajajajajar
1773                                                  asi
Name: clean_text, Length: 1774, dtype: object

In [30]:
# Función para generar n-grams de una lista de tokens
def generar_ngrams(tokens, n):
    return list(ngrams(tokens, n))

# Tokenización y generación de n-grams para cada fila del DataFrame
n = 2  # Tamaño de los n-grams

def analisis_sentimiento(texto):
    # Tokenizar el texto
    tokens = word_tokenize(texto)
    # Generar n-grams
    n_grams = generar_ngrams(tokens, n)
    # Verificar si hay n-grams
    if not n_grams:
        return 'neutro'  # Devolver 'neutro' si no hay n-grams
    # Calcular el sentimiento para cada n-gram y promediar los resultados
    sentimientos = [TextBlob(' '.join(ngram)).sentiment.polarity for ngram in n_grams]
    # Calcular el promedio de los sentimientos de los n-grams
    promedio_sentimientos = sum(sentimientos) / len(sentimientos)
    # Categorizar el sentimiento
    if promedio_sentimientos > 0.2:
        return 'positivo'
    elif promedio_sentimientos < -0.2:
        return 'negativo'
    else:
        return 'neutro'

# Aplicar el análisis de sentimientos a cada texto en el DataFrame
df['n-grams'] = df['clean_text'].apply(analisis_sentimiento)

df

,Fecha,Dia,Num_Day,Num_Month,Month,Year,Hora,Persona,Mensaje,clean_text,timestamp,texBlod,sentimiento_vader,n-grams
0,30/11/2023,Jueves,30,11,Nov,2023,19:12:50,Capibaras Ironhack,‎Los mensajes y las llamadas están cifrados de...,mensaje llamada cifrado extremo extremo nadie ...,2023-11-30 19:12:50,neutro,neutro,neutro
1,30/11/2023,Jueves,30,11,Nov,2023,19:12:50,Alana Ironhack,‎Alana Ironhack creó el grupo “Capibaras Ironh...,aln ironhack creer grupo capibara ironhack,2023-11-30 19:12:50,neutro,neutro,neutro
2,30/11/2023,Jueves,30,11,Nov,2023,19:12:50,Capibaras Ironhack,‎Alana Ironhack te añadió.,aln ironhack t anadio,2023-11-30 19:12:50,neutro,neutro,neutro
3,30/11/2023,Jueves,30,11,Nov,2023,19:12:56,Alana Ironhack,Hola,holar,2023-11-30 19:12:56,neutro,neutro,neutro
4,30/11/2023,Jueves,30,11,Nov,2023,19:13:12,Alana Ironhack,Les paso las fotos de ayer prepárense xq es un...,l pasar foto ayer preparense ser bombardeo,2023-11-30 19:13:12,neutro,neutro,neutro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1769,17/02/2024,Sabado,17,2,Feb,2024,21:09:53,Sil🐨,No y que también pensaba que esos días que íba...,tambien pensar dia ibir hacer,2024-02-17 21:09:53,neutro,neutro,neutro
1770,17/02/2024,Sabado,17,2,Feb,2024,21:11:04,Alana Ironhack,Jajaj,jajaj,2024-02-17 21:11:04,neutro,neutro,neutro
1771,17/02/2024,Sabado,17,2,Feb,2024,21:11:18,Alana Ironhack,Disfrutar de tu libertad,disfrutar libertad,2024-02-17 21:11:18,neutro,neutro,neutro
1772,17/02/2024,Sabado,17,2,Feb,2024,21:18:17,Sil🐨,JAJAJAJAJA,jajajajajar,2024-02-17 21:18:17,neutro,neutro,neutro


# spacy = no se puede en español

In [31]:
#!pip install spacy
#!python -m spacy download en_core_web_sm

import spacy
from textblob import TextBlob
import spacy
from spacy.lang.es.examples import sentences 

In [32]:
sentences = ', '.join(df['clean_text'])


In [35]:
nlp = spacy.load("es_core_news_sm")
doc = nlp(sentences)

In [36]:
# Analyze syntax
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

Noun phrases: ['mensaje llamada cifrado extremo extremo', 'nadie chat siquiera', 'l', 'aln ironhack', 'grupo capibara ironhack, aln ironhack t anadio, holar,', 'foto', 'practicamente laboratorio', ', jajaja', ', gracia', ', fan icono grupo', 'laboratorio ultimo', 'ejercicio facetgrid', 'respuesta rosella', 'l', ', bueno rosella', 'buen trabajo nena', 'enferma', ', cierto final', 'gripe', 'habertelo', 'pegado joo', 'l', ', metro veo mundo malo,', 'juancho', 'final', 'ser verdad, cancelado enfermedad l edito mensaje,', 'ahi frio', 'david', 'ira', 'cada casa', 'ratito', ', valee perfecto', 'forma', 'conecto', 'plot', 'siiii iplot', 'l', 'casa', 'adema querio', 'muchisima tos', 'recuperate', ', conectare', 'estare atento', 'linda', 'mil gracia', 'habia', ', ohhh monar', 'jajaja', 'l peor curso', 'garabato', 'cualquiera hoja', 'aburr', 'arte', 'l', 'video bueno', 'eda', 'dos mmm', 'vago', 'momento', 'pereza', ', tomalo festivo', 'video', 'vida', 'l', 'eda', 'l', 'paso', 'cada hora', 'faring

In [37]:
# Find named entities, phrases and concepts
for entity in doc.ents:
    print(entity.text, entity.label_)

joo MISC
david LOC
eh ORG
biblioteca cufflinks LOC
ayyy siiii PER
adema ORG
octavio PER
awww linda mil gracia PER
capibara LOC
l eda l PER
guay LOC
ansia PER
graficaste l dado PER
monton cosa PER
catgpt ORG
oir MISC
def median ORG
half len PER
lsort if not len l return l ORG
print median lst PER
alejo mann PER
jajajaj LOC
httpstowardsdatasciencecom creatingboxplotsofwelllogdatausingmatplotlibinpythoncef PER
capibara LOC
ole guay LOC
ash PER
l l comar PER
jajaja LOC
sql MISC
capis mann PER
rrhh despu comer PER
l anoche comenzo PER
mann PER
mann PER
awww PER
jajajajajar PER
tenio mal PER
awww bolitar lindo PER
sql MISC
sql MISC
valee risar PER
victor l PER
sql MISC
sql MISC
mann PER
github MISC
llevais LOC
sql MISC
sql MISC
guay LOC
vacacin PER
vacacin PER
vacacin nosotra PER
sql MISC
sql MISC
senor entendar PER
capiiis PER
jajajaj PER
aqui perdi PER
dyson wrap PER
mac MISC
peinser PER
mac MISC
dyson jajar PER
windows MISC
vera l lograr LOC
sii l pensar MISC
sql MISC
python MISC
sql MISC